# Evolving Emission Measure Distributions
Look at the emission measure distribution ($\mathrm{EM}(T)$) as a function of time for a given run, for both a uniform and power-law case, comparing equilibrium and non-equilibrium results. Choose an intermediate frequency, say around $t_N\approx2000$ s to show some interesting results.

In [1]:
import os,sys
sys.path.append(os.path.join(os.environ['RESEARCH_DIR'],'EBTEL_analysis/src'))
from random import randint
import numpy as np
import matplotlib.pyplot as plt
if plt.get_backend()=='MacOSX':
    plt.switch_backend('Qt4Agg')
print(plt.get_backend())
import matplotlib.gridspec as gridspec
from matplotlib import animation
import seaborn.apionly as sns
import em_binner as emb
#%matplotlib inline

MacOSX


## Data Selection
Grab the data we want from our database. For the power-law case, we'll start off with a dataset where the waiting time is **not** proportional to the event energy.

In [2]:
em_results_dict = {}
tn_sample_val = 2500
#uniform
data = np.loadtxt('em_movie_data/alphauniform/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.txt'%(tn_sample_val))#,tn_sample_val))
em_results_dict['uniform_ieq'] = {'t':data[:,0],'T':data[:,1],'n':data[:,3],'q':data[:,15]}
data = np.loadtxt('em_movie_data/alphauniform/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.ips_results.txt'%(tn_sample_val))#,tn_sample_val))
em_results_dict['uniform_nei'] = {'t':data[:,0],'T':data[:,2],'n':data[:,3]}
#power-law
data = np.loadtxt('em_movie_data/alpha2.5/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.txt'%(tn_sample_val))#,tn_sample_val))
em_results_dict['pl_ieq'] = {'t':data[:,0],'T':data[:,1],'n':data[:,3],'q':data[:,15]}
data = np.loadtxt('em_movie_data/alpha2.5/ebtel_L40.0_tn%d_tpulse200.0_rka4_0.ips_results.txt'%(tn_sample_val))#,tn_sample_val))
em_results_dict['pl_nei'] = {'t':data[:,0],'T':data[:,2],'n':data[:,3]}
#power-law with scaling
data = np.loadtxt('em_movie_data/alpha2.5/ebtel_L40.0_tn%d-b1.0_tpulse200.0_rka4_0.txt'%(tn_sample_val))#,tn_sample_val))
em_results_dict['pl_beta_ieq'] = {'t':data[:,0],'T':data[:,1],'n':data[:,3],'q':data[:,15]}
data = np.loadtxt('em_movie_data/alpha2.5/ebtel_L40.0_tn%d-b1.0_tpulse200.0_rka4_0.ips_results.txt'%(tn_sample_val))#,tn_sample_val))
em_results_dict['pl_beta_nei'] = {'t':data[:,0],'T':data[:,2],'n':data[:,3]}

## Emission Measure Calculation
Calculate the emission measure at each timestep for each of the datasets.

In [3]:
em_maker = emb.EM_Binner(2.*40.0e+8)

In [4]:
for key in em_results_dict:
    em_maker.set_data(em_results_dict[key]['t'],em_results_dict[key]['T'],em_results_dict[key]['n'])
    em_maker.build_em_dist(build_mat=True)
    em_results_dict[key]['em_mat'] = em_maker.em_mat
    em_results_dict[key]['T_em'] = em_maker.T_em

## Plotting $\mathrm{EM}(T)$ versus time, $t$
Try overlaying all of the plots on each other.

### Make a movie (needs help)

In [5]:
def make_em_movie(ieq_data,nei_data, interval=20, fps=30, movie_filename=None,duration='full'):
    """Make movie of evolving EM distribution"""
    
    if duration=='full':
        duration=ieq_data['t']
        duration_ind = len(duration)
    else:
        duration_ind=duration
        duration=ieq_data['t'][0:duration]
    
    #set up the figure
    fig = plt.figure(figsize=(12,10))
    gs = gridspec.GridSpec(4,6)
    ax_heat = fig.add_subplot(gs[0,:])
    ax_t = fig.add_subplot(gs[1,:])
    ax_n = ax_t.twinx()
    ax_snap = fig.add_subplot(gs[2:,0:2])
    ax_ieq = fig.add_subplot(gs[2:,2:4])
    ax_nei = fig.add_subplot(gs[2:,4:])
    
    #set limits
    ax_heat.set_xlim([duration[0],duration[-1]])
    ax_heat.set_ylim([0,np.max(ieq_data['q'][0:duration_ind])])
    ax_t.set_xlim([duration[0],duration[-1]])
    ax_t.set_ylim([0,np.max(ieq_data['T'][0:duration_ind])/1e+6])
    ax_n.set_xlim([duration[0],duration[-1]])
    ax_n.set_ylim([0,np.max(ieq_data['n'][0:duration_ind])/1e+8])
    ax_snap.set_xlim([10**5.5,10**7.5])
    ax_snap.set_ylim([10**27.,10**30.])
    ax_ieq.set_xlim([10**5.5,10**7.5])
    ax_ieq.set_ylim([10**27.,10**30.])
    ax_nei.set_xlim([10**5.5,10**7.5])
    ax_nei.set_ylim([10**27.,10**30.])
    
    #labels and ticks and scale
    ax_heat.set_xlabel(r'$t\,\,\mathrm{(s)}$',fontsize=22)
    ax_t.set_xlabel(r'$t\,\,\mathrm{(s)}$',fontsize=22)
    ax_t.set_ylabel(r'$T\,\,\mathrm{(MK)}$',fontsize=22)
    ax_n.set_ylabel(r'$n\,\,(10^8\,\mathrm{cm}^{-3})$',fontsize=22)
    ax_heat.set_ylabel(r'$H\,\,(\mathrm{erg}\,\mathrm{cm}^{-3}\,\mathrm{s}^{-1})$',fontsize=22)
    ax_snap.set_xlabel(r'$T\,\,\mathrm{(K)}$',fontsize=22)
    ax_ieq.set_xlabel(r'$T\,\,\mathrm{(K)}$',fontsize=22)
    ax_nei.set_xlabel(r'$T\,\,\mathrm{(K)}$',fontsize=22)
    ax_snap.set_ylabel(r'$\mathrm{EM}\,\,(\mathrm{cm}^{-5})$',fontsize=22)
    ax_ieq.tick_params(axis='y',labelleft='off')
    ax_nei.tick_params(axis='y',labelleft='off')
    ax_snap.set_yscale('log')
    ax_snap.set_xscale('log')
    ax_ieq.set_yscale('log')
    ax_ieq.set_xscale('log')
    ax_nei.set_yscale('log')
    ax_nei.set_xscale('log')
    
    #plot the heating
    ax_heat.plot(duration,ieq_data['q'][0:duration_ind],linewidth=2,color=sns.color_palette('deep')[0])
    ax_t.plot(duration,ieq_data['T'][0:duration_ind]/1e+6,linewidth=2,color=sns.color_palette('deep')[0])
    ax_n.plot(duration,ieq_data['n'][0:duration_ind]/1e+8,linewidth=2,color=sns.color_palette('deep')[2])
                     
    plt.tight_layout()

    #setup the plotters
    progress, = ax_heat.plot([],[],linewidth=2,color='k',linestyle='--')
    progress_t, = ax_t.plot([],[],linewidth=2,color='k',linestyle='--')
    em_ieq_snap, = ax_snap.plot([],[],linestyle='-',color=sns.color_palette('deep')[0],linewidth=2)
    em_nei_snap, = ax_snap.plot([],[],linestyle='--',color=sns.color_palette('deep')[2],linewidth=2)
    em_snap_ieq, = ax_ieq.plot([],[],linestyle='-',color=sns.color_palette('deep')[0],linewidth=2)
    em_avg_ieq, = ax_ieq.plot([],[],linestyle='--',color=sns.color_palette('deep')[2],linewidth=2)
    em_snap_nei, = ax_nei.plot([],[],linestyle='-',color=sns.color_palette('deep')[0],linewidth=2)
    em_avg_nei, = ax_nei.plot([],[],linestyle='--',color=sns.color_palette('deep')[2],linewidth=2)
    
    #define initializer
    def init():
        progress.set_data([],[])
        progress_t.set_data([],[])
        em_ieq_snap.set_data([],[])
        em_nei_snap.set_data([],[])
        em_snap_ieq.set_data([],[])
        em_avg_ieq.set_data([],[])
        em_snap_nei.set_data([],[])
        em_avg_nei.set_data([],[])
        return progress,progress_t,em_ieq_snap,em_nei_snap,em_snap_ieq,em_avg_ieq,em_snap_nei,em_avg_nei
    
    #define animator function
    def animate(i):
        #get data and compute running averages
        em_i = ieq_data['em_mat'][i,:]
        em_n = nei_data['em_mat'][i,:]
        if i > 0:
            em_i_ra = np.average(ieq_data['em_mat'][0:i+1,:],weights=np.gradient(ieq_data['t'][0:i+1]),axis=0)
            em_n_ra = np.average(nei_data['em_mat'][0:i+1,:],weights=np.gradient(nei_data['t'][0:i+1]),axis=0)
        else:
            em_i_ra = em_i
            em_n_ra = em_n
        #set data
        progress.set_data([ieq_data['t'][i],ieq_data['t'][i]],[0.0,np.max(ieq_data['q'])])
        progress_t.set_data([ieq_data['t'][i],ieq_data['t'][i]],[0.0,np.max(ieq_data['T'])/1e+6])
        em_ieq_snap.set_data(ieq_data['T_em'],em_i)
        em_nei_snap.set_data(nei_data['T_em'],em_n) 
        em_snap_ieq.set_data(ieq_data['T_em'],em_i)
        em_avg_ieq.set_data(ieq_data['T_em'],em_i_ra)
        em_snap_nei.set_data(nei_data['T_em'],em_n)
        em_avg_nei.set_data(nei_data['T_em'],em_n_ra)
        return progress,progress_t,em_ieq_snap,em_nei_snap,em_snap_ieq,em_avg_ieq,em_snap_nei,em_avg_nei
    
    #make animation
    anim = animation.FuncAnimation(fig,animate,init_func=init, frames=len(duration), blit=False, interval=interval )

    #save or return animation handler
    if movie_filename is not None:
        metadata={'title':'Evolving EM distributions, IEQ and NEI','author':'Will Barnes'}
        anim.save(movie_filename,writer=animation.writers['ffmpeg'](fps=fps,metadata=metadata,bitrate=1800))
    else:
        plt.show()
        #plt.close(anim._fig)
        #return anim
    

In [7]:
make_em_movie(em_results_dict['uniform_ieq'],em_results_dict['uniform_nei'],interval=10,)#duration=1200,movie_filename='/home/wtb2/Desktop/emEvolve_tn%d_uniform.mp4'%(tn_sample_val))

/Users/willbarnes/anaconda/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [8]:
make_em_movie(em_results_dict['pl_ieq'],em_results_dict['pl_nei'],interval=10,)#duration=1200,movie_filename='/home/wtb2/Desktop/emEvolve_tn%d_pl.mp4'%(tn_sample_val))

In [9]:
make_em_movie(em_results_dict['pl_beta_ieq'],em_results_dict['pl_beta_nei'],interval=10,)#duration=1200,movie_filename='/home/wtb2/Desktop/emEvolve_tn%d_plBeta.mp4'%(tn_sample_val))